In [1]:
import sys
sys.executable

'C:\\Users\\Isaac\\Documents\\miniconda3\\envs\\3dvision\\python.exe'

In [7]:
import argparse
import math
import h5py
import numpy as np
import tensorflow as tf
import socket

import os
import sys
BASE_DIR = os.path.dirname(os.getcwd())
BASE_DIR = os.path.join(BASE_DIR, "Pointnet_Semantic_Segmentation" )
sys.path.append(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'utils'))
sys.path.append(os.path.join(BASE_DIR, 'data'))
import provider
import tf_util
from model import *


In [8]:
BATCH_SIZE = 24
NUM_POINT = 4096
MAX_EPOCH = 50
BASE_LEARNING_RATE = 0.001
GPU_INDEX = 0
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 300000
DECAY_RATE = 0.5

LOG_DIR = 'log'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp model.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp train.py %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

MAX_NUM_POINT = 4096
NUM_CLASSES = 13

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
#BN_DECAY_DECAY_STEP = float(DECAY_STEP * 2)
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

ALL_FILES = provider.getDataFiles('./data/indoor3d_sem_seg_hdf5_data/all_files.txt')
room_filelist = [line.rstrip() for line in open('./data/indoor3d_sem_seg_hdf5_data/room_filelist.txt')]


In [9]:
ALL_FILES

['indoor3d_sem_seg_hdf5_data/ply_data_all_0.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_1.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_2.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_3.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_4.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_5.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_6.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_7.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_8.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_9.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_10.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_11.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_12.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_13.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_14.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_15.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_16.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_17.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_18.h5',
 'indoor3d_sem_seg_hdf5_data/ply_data_all_19.h5',
 'indoor3d

In [13]:
# Load ALL data
data_batch_list = []
label_batch_list = []
for h5_filename in ALL_FILES:
    data_batch, label_batch = provider.loadDataFile("./data/"+h5_filename)
    data_batch_list.append(data_batch)
    label_batch_list.append(label_batch)
data_batches = np.concatenate(data_batch_list, 0)
label_batches = np.concatenate(label_batch_list, 0)
print(data_batches.shape)
print(label_batches.shape)

(23585, 4096, 9)
(23585, 4096)


In [19]:
features = ["x","y","z","r","g","b","xn","yn","zn"]
for i in range(9): 
    print(features[i] + "_range :", np.min(data_batch[:, :, i]), np.max(data_batch[:, :, i]))

x_range : -0.5 0.5
y_range : -0.5 0.5
z_range : 0.0 2.94
r_range : 0.0 1.0
g_range : 0.0 1.0
b_range : 0.0 1.0
xn_range : 0.0 1.0
yn_range : 0.0 1.0
zn_range : 0.0 1.0


In [14]:
room_filelist

['Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceRoom_1',
 'Area_1_conferenceR

In [15]:
len(room_filelist)

23585

In [16]:
test_area = 'Area_'+str(6)
train_idxs = []
test_idxs = []
for i,room_name in enumerate(room_filelist):
    if test_area in room_name:
        test_idxs.append(i)
    else:
        train_idxs.append(i)

## seperate train and test data
train_data = data_batches[train_idxs,...]
train_label = label_batches[train_idxs]
test_data = data_batches[test_idxs,...]
test_label = label_batches[test_idxs]

print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(20291, 4096, 9) (20291, 4096)
(3294, 4096, 9) (3294, 4096)


In [ ]:

def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)


def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def train():
    with tf.Graph().as_default():
        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())
            
            # Note the global_step=batch parameter to minimize. 
            # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
            batch = tf.Variable(0)
            bn_decay = get_bn_decay(batch)
            tf.summary.scalar('bn_decay', bn_decay)

            # Get model and loss 
            pred = get_model(pointclouds_pl, is_training_pl, bn_decay=bn_decay)
            loss = get_loss(pred, labels_pl)
            tf.summary.scalar('loss', loss)

            correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
            accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
            tf.summary.scalar('accuracy', accuracy)

            # Get training operator
            learning_rate = get_learning_rate(batch)
            tf.summary.scalar('learning_rate', learning_rate)
            if OPTIMIZER == 'momentum':
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
            elif OPTIMIZER == 'adam':
                optimizer = tf.train.AdamOptimizer(learning_rate)
            train_op = optimizer.minimize(loss, global_step=batch)
            
            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()
            
        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = True
        sess = tf.Session(config=config)

        # Add summary writers
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'))

        # Init variables
        init = tf.global_variables_initializer()
        sess.run(init, {is_training_pl:True})

        ops = {'pointclouds_pl': pointclouds_pl,
               'labels_pl': labels_pl,
               'is_training_pl': is_training_pl,
               'pred': pred,
               'loss': loss,
               'train_op': train_op,
               'merged': merged,
               'step': batch}

        for epoch in range(MAX_EPOCH):
            log_string('**** EPOCH %03d ****' % (epoch))
            sys.stdout.flush()
             
            train_one_epoch(sess, ops, train_writer)
            eval_one_epoch(sess, ops, test_writer)
            
            # Save the variables to disk.
            if epoch % 10 == 0:
                save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
                log_string("Model saved in file: %s" % save_path)



def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    current_data, current_label, _ = provider.shuffle_data(train_data[:,0:NUM_POINT,:], train_label) 
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        if batch_idx % 100 == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred']],
                                         feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
    
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))

        
def eval_one_epoch(sess, ops, test_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    current_data = test_data[:,0:NUM_POINT,:]
    current_label = np.squeeze(test_label)
    
    file_size = current_data.shape[0]
    num_batches = file_size // BATCH_SIZE
    
    for batch_idx in range(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['labels_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}
        summary, step, loss_val, pred_val = sess.run([ops['merged'], ops['step'], ops['loss'], ops['pred']],
                                      feed_dict=feed_dict)
        test_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += (loss_val*BATCH_SIZE)
        for i in range(start_idx, end_idx):
            for j in range(NUM_POINT):
                l = current_label[i, j]
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i-start_idx, j] == l)
            
    log_string('eval mean loss: %f' % (loss_sum / float(total_seen/NUM_POINT)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
         

if __name__ == "__main__":
    train()
    LOG_FOUT.close()

Tensor("fc2/Relu:0", shape=(24, 128), dtype=float32, device=/device:GPU:0)
**** EPOCH 000 ****
----
Current batch/total batch num: 0/845
Current batch/total batch num: 100/845
Current batch/total batch num: 200/845
Current batch/total batch num: 300/845
Current batch/total batch num: 400/845
Current batch/total batch num: 500/845
Current batch/total batch num: 600/845
Current batch/total batch num: 700/845
Current batch/total batch num: 800/845
mean loss: 0.702116
accuracy: 0.772028
----
eval mean loss: 0.596886
eval accuracy: 0.796427
eval avg class acc: 0.580015
Model saved in file: log\model.ckpt
**** EPOCH 001 ****
----
Current batch/total batch num: 0/845
Current batch/total batch num: 100/845
Current batch/total batch num: 200/845
Current batch/total batch num: 300/845
Current batch/total batch num: 400/845
Current batch/total batch num: 500/845
Current batch/total batch num: 600/845
Current batch/total batch num: 700/845
Current batch/total batch num: 800/845
mean loss: 0.509343